In [1]:
import pandas as pd
pd.set_option('display.max_rows', 15)
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import operator

In [208]:
data = pd.read_csv('./nba_data.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data_train = pd.DataFrame(data[data.Year < 2017])
data_test = pd.DataFrame(data[data.Year == 2017])
data['Champion'] = data['Champion'] * 1
data.set_value(1312, 'Champion', 1)

,Team,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,...,2P%,FT%,RPG,APG,STL,BLK,TOVPG,PPG,Accolades,Franchise ID
0,Boston Celtics,1960.0,59.0,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.417,0.734,80.2,24.7,0.0,0.0,0.0,124.5,13.0,Celtics
1,Golden State Warriors,1960.0,49.0,0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.409,0.669,78.9,23.9,0.0,0.0,0.0,118.6,9.0,Warriors
2,Philadelphia 76ers,1960.0,45.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.414,0.791,72.1,22.3,0.0,0.0,0.0,118.9,5.0,76ers
3,New York Knicks,1960.0,27.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.421,0.765,70.0,22.2,0.0,0.0,0.0,117.3,5.0,Knicks
4,Atlanta Hawks,1960.0,46.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.419,0.745,71.2,25.1,0.0,0.0,0.0,113.4,10.0,Hawks
5,Detroit Pistons,1960.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.397,0.729,73.2,19.6,0.0,0.0,0.0,111.6,3.0,Pistons
6,Los Angeles Lakers,1960.0,25.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.386,0.730,72.4,19.3,0.0,0.0,0.0,107.3,7.0,Lakers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,Denver Nuggets,2017.0,40.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.519,0.774,46.4,25.3,6.9,4.0,14.9,111.7,1.0,Nuggets
1321,New Orleans Pelicans,2017.0,34.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.495,0.750,43.7,22.8,7.8,5.5,12.9,104.3,5.0,Pelicans


In [202]:
data

,Team,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,...,2P%,FT%,RPG,APG,STL,BLK,TOVPG,PPG,Accolades,Franchise ID
0,Boston Celtics,1960.0,59.0,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.417,0.734,80.2,24.7,0.0,0.0,0.0,124.5,13.0,Celtics
1,Golden State Warriors,1960.0,49.0,0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.409,0.669,78.9,23.9,0.0,0.0,0.0,118.6,9.0,Warriors
2,Philadelphia 76ers,1960.0,45.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.414,0.791,72.1,22.3,0.0,0.0,0.0,118.9,5.0,76ers
3,New York Knicks,1960.0,27.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.421,0.765,70.0,22.2,0.0,0.0,0.0,117.3,5.0,Knicks
4,Atlanta Hawks,1960.0,46.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.419,0.745,71.2,25.1,0.0,0.0,0.0,113.4,10.0,Hawks
5,Detroit Pistons,1960.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.397,0.729,73.2,19.6,0.0,0.0,0.0,111.6,3.0,Pistons
6,Los Angeles Lakers,1960.0,25.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.386,0.730,72.4,19.3,0.0,0.0,0.0,107.3,7.0,Lakers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,Denver Nuggets,2017.0,40.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.519,0.774,46.4,25.3,6.9,4.0,14.9,111.7,1.0,Nuggets
1321,New Orleans Pelicans,2017.0,34.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.495,0.750,43.7,22.8,7.8,5.5,12.9,104.3,5.0,Pelicans


In [4]:
inputs = ['Wins', 'MVP', 'Scoring Leader', 'Rebound Leader', 
          'Assist Leader', 'WS Leader', 'DPOY', 'MIP', '6MOY', 
          'Coach of Year', 'All-Stars', 'All-Defensive', 'All-NBA', 
          'FG%', '3P Attempts', '3P%', '2P Attempts', '2P%', 'FT%',
          'RPG', 'APG', 'STL', 'BLK', 'TOVPG', 'PPG', 'Accolades', ]

for inp in inputs:
    data_train[inp] = preprocessing.scale(data_train[inp])
    data_test[inp] = preprocessing.scale(data_test[inp])


In [ ]:
model = LogisticRegression()
inputs = ['Wins', 'MVP', 'Scoring Leader', 'Rebound Leader', 
          'Assist Leader', 'WS Leader', 'DPOY', 'MIP', '6MOY', 
          'Coach of Year', 'All-Stars', 'All-Defensive', 'All-NBA', 
          'FG%', '3P Attempts', '3P%', '2P Attempts', '2P%', 'FT%',
          'RPG', 'APG', 'STL', 'BLK', 'TOVPG', 'PPG', 'Accolades', ]

model.fit(data_train[inputs], data_train['Champion'])
model.coef_

It looks like having the scoring leader, the sixth man of the year, and coach of the year greatly increase your chance of winning the finals. 

In [109]:
data_train.groupby('Year')[inputs]

In [ ]:
model = RandomForestClassifier()

model.fit(data[inputs], data['Champion'])

In [ ]:
model.predict_proba(data[inputs])

In [ ]:
data

In [ ]:
data[data.Team == 'Cleveland Cavaliers']

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
asd = ['Accolades','PPG','RPG','APG']
model.fit(data[asd], data['Wins'])

In [ ]:
data['pred_wins'] = model.predict(data[asd])

In [ ]:
data

In [ ]:
print(-cross_val_score(model, data[inputs], data['Wins'], cv= 10,
                       scoring='neg_mean_squared_error').mean())

In [ ]:
(data['pred_wins'] - data['Wins']).mean()

In [ ]:
(data['pred_wins'] - data['Wins']).std()

In [5]:
elo_data = pd.read_csv('./nbaallelo.csv')
elo_data = elo_data[elo_data.year_id > 1958]

In [6]:
elo_data

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
8460,4231,195810190SYR,NBA,1,1959,10/19/1958,1,0,DET,Pistons,...,37.834084,SYR,Sixers,103,1553.9590,1558.7543,A,L,0.278847,NaN
8461,4231,195810190SYR,NBA,0,1959,10/19/1958,1,0,SYR,Sixers,...,45.638824,DET,Pistons,94,1488.8943,1484.0990,H,W,0.721153,NaN
8462,4232,195810220LAL,NBA,0,1959,10/22/1958,1,0,MNL,Lakers,...,30.401035,CIN,Kings,79,1481.4917,1465.9398,H,W,0.520005,NaN
8463,4232,195810220LAL,NBA,1,1959,10/22/1958,1,0,CIN,Kings,...,35.948742,MNL,Lakers,99,1395.4003,1410.9521,A,L,0.479995,NaN
8464,4233,195810230STL,NBA,1,1959,10/23/1958,2,0,DET,Pistons,...,37.622662,STL,Hawks,104,1545.3391,1547.3676,A,L,0.283296,NaN
8465,4233,195810230STL,NBA,0,1959,10/23/1958,1,0,STL,Hawks,...,44.456135,DET,Pistons,103,1484.0990,1482.0706,H,W,0.716704,NaN
8466,4234,195810250CIN,NBA,0,1959,10/25/1958,2,0,CIN,Kings,...,36.701500,MNL,Lakers,94,1410.9521,1403.6812,H,W,0.709340,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126307,63154,201506090CLE,NBA,0,2015,6/9/2015,99,1,CLE,Cavaliers,...,61.826408,GSW,Warriors,91,1797.5032,1790.9591,H,W,0.527843,NaN
126308,63155,201506110CLE,NBA,1,2015,6/11/2015,101,1,GSW,Warriors,...,67.794983,CLE,Cavaliers,82,1723.4149,1704.3949,A,W,0.453428,NaN


In [7]:
print(elo_data[(elo_data.seasongame < 82) & (elo_data.fran_id == 'Celtics')].groupby('year_id')['pts'].mean())
elo_data[(elo_data.fran_id == 'Celtics')].groupby('year_id')['pts'].mean()

year_id
1959    117.209877
1960    123.654321
1961    119.580247
1962    121.037037
1963    118.962963
1964    112.901235
1965    112.740741
           ...    
2009    100.740741
2010     99.271605
2011     96.308642
2012     91.320988
2013     96.518519
2014     96.172840
2015    101.320988
Name: pts, dtype: float64


year_id
1959    117.289157
1960    122.818182
1961    119.853933
1962    120.170213
1963    118.698925
1964    111.955556
1965    113.315217
           ...    
2009    101.093750
2010     98.009434
2011     96.318681
2012     91.302326
2013     95.540230
2014     96.243902
2015    101.058140
Name: pts, dtype: float64

In [8]:
teams = list(elo_data['fran_id'].unique())

In [9]:
diffs = {}
for i in range(1,82):
    diffs[i] = []
for team in teams:
    for i in range(1,82):
        diffs[i].append(((elo_data[(elo_data.seasongame <= i) & (elo_data.fran_id == team) & (elo_data.is_playoffs == 0)].groupby('year_id')['pts'].mean()
        - elo_data[(elo_data.fran_id == team) & (elo_data.is_playoffs == 0)].groupby('year_id')['pts'].mean()).mean()))

In [19]:
for i in range(1,82):
    diffs[i] = np.array(diffs[i]).mean()
    

In [20]:
# These are the differences between season average ppg and average ppg through n games. 
diffs

{1: -1.2434451111605078,
 2: -1.3969480303558133,
 3: -1.2481851293783479,
 4: -1.3592867421369026,
 5: -1.3203884345102039,
 6: -1.2638428955998389,
 7: -1.2163837778497903,
 8: -1.2329758664081687,
 9: -1.2563557569493486,
 10: -1.2237611653031657,
 11: -1.203367094732563,
 12: -1.1820289198458631,
 13: -1.1161772318027152,
 14: -1.0665274373830016,
 15: -1.0070322682334383,
 16: -0.98484258607061381,
 17: -0.95626285174873726,
 18: -0.93974349383846756,
 19: -0.94815676079272426,
 20: -0.9517483012313821,
 21: -0.95260982564790264,
 22: -0.94855530957392575,
 23: -0.94770894635581537,
 24: -0.96040864999658926,
 25: -0.96501125875541971,
 26: -0.96121992860794381,
 27: -0.95310472266874602,
 28: -0.93441438686514289,
 29: -0.90802461219935071,
 30: -0.89800419883297911,
 31: -0.90846796109805872,
 32: -0.90408904705963655,
 33: -0.88994466017982832,
 34: -0.87707273325724588,
 35: -0.83735936024835578,
 36: -0.82396455335514407,
 37: -0.77408257318429086,
 38: -0.75680506217450239,


In [ ]:
elo_data.columns

In [ ]:
elo_data[(elo_data.seasongame >= 42) & (elo_data.fran_id == 'Celtics') & (elo_data.is_playoffs == 0)].groupby('year_id')['pts'].mean()

In [ ]:
data_test

In [ ]:
data_t = data[data.Year <2016]

In [21]:
model = RandomForestClassifier()
model.fit(data_train[inputs], data_train['Champion'])
model.predict_proba(data_test[inputs])

array([[ 0.9,  0.1],
       [ 0.8,  0.2],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 0.8,  0.2],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 0.8,  0.2],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 0.9,  0.1],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 0.5,  0.5],
       [ 0.9,  0.1],
       [ 0.8,  0.2],
       [ 0.9,  0.1],
       [ 1. ,  0. ],
       [ 0.8,  0.2],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 0.9,  0.1],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 0.8,  0.2]])

In [ ]:
data_fff = data[data.Year == 2016]
data_fff.drop('Champion',axis=1, inplace=True)
data_fff

In [ ]:
model = RandomForestClassifier()
model.fit(data_train[inputs], data_train['Champion'])

In [ ]:
model.predict_proba(data_fff[inputs])

In [227]:
elo_data = pd.read_csv('./nbaallelo.csv')

In [228]:
elo_data

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,40.294830,NYK,Knicks,68,1300.0000,1306.7233,H,L,0.640065,NaN
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,41.705170,TRH,Huskies,66,1300.0000,1293.2767,A,W,0.359935,NaN
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,42.012257,NYK,Knicks,47,1306.7233,1297.0712,H,W,0.631101,NaN
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,40.692783,CHS,Stags,63,1300.0000,1309.6521,A,L,0.368899,NaN
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,38.864048,WSC,Capitols,50,1300.0000,1320.3811,H,L,0.640065,NaN
5,3,194611020DTF,NBA,1,1947,11/2/1946,1,0,WSC,Capitols,...,43.135952,DTF,Falcons,33,1300.0000,1279.6189,A,W,0.359935,NaN
6,4,194611020PRO,NBA,1,1947,11/2/1946,1,0,BOS,Celtics,...,40.459381,PRO,Steamrollers,59,1300.0000,1305.1542,A,L,0.359935,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126307,63154,201506090CLE,NBA,0,2015,6/9/2015,99,1,CLE,Cavaliers,...,61.826408,GSW,Warriors,91,1797.5032,1790.9591,H,W,0.527843,NaN
126308,63155,201506110CLE,NBA,1,2015,6/11/2015,101,1,GSW,Warriors,...,67.794983,CLE,Cavaliers,82,1723.4149,1704.3949,A,W,0.453428,NaN


In [30]:
data[data.Year == 2016]['Team'].unique()

array(['Cleveland Cavaliers', 'Toronto Raptors', 'Miami Heat',
       'Atlanta Hawks', 'Boston Celtics', 'Charlotte Hornets',
       'Indiana Pacers', 'Detroit Pistons', 'Chicago Bulls',
       'Washington Wizards', 'Orlando Magic', 'Milwaukee Bucks',
       'New York Knicks', 'Brooklyn Nets', 'Philadelphia 76ers',
       'Golden State Warriors', 'San Antonio Spurs',
       'Oklahoma City Thunder', 'Los Angeles Clippers',
       'Portland Trail Blazers', 'Dallas Mavericks', 'Memphis Grizzlies',
       'Houston Rockets', 'Utah Jazz', 'Sacramento Kings',
       'Denver Nuggets', 'New Orleans Pelicans', 'Minnesota Timberwolves',
       'Phoenix Suns', 'Los Angeles Lakers'], dtype=object)

In [64]:
model = RandomForestClassifier()

model.fit(data[data.Year < 2017][inputs], data[data.Year < 2017]['Champion'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [74]:
max(model.predict_proba(data_test[inputs])[1])

0.5

In [70]:
data_test.iloc[1]

Team              Cleveland Cavaliers
Year                             2017
Wins                               51
Champion                        False
MVP                                 0
Scoring Leader                      0
Rebound Leader                      0
                         ...         
APG                              22.7
STL                               6.6
BLK                                 4
TOVPG                            13.7
PPG                             110.3
Accolades                           6
Franchise ID                Cavaliers
Name: 1298, dtype: object

In [82]:
data['pred'] = model.predict(data[inputs])

In [83]:
data

,Team,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,...,FT%,RPG,APG,STL,BLK,TOVPG,PPG,Accolades,Franchise ID,pred
0,Boston Celtics,1960.0,59.0,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.734,80.2,24.7,0.0,0.0,0.0,124.5,13.0,Celtics,1
1,Golden State Warriors,1960.0,49.0,0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.669,78.9,23.9,0.0,0.0,0.0,118.6,9.0,Warriors,0
2,Philadelphia 76ers,1960.0,45.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.791,72.1,22.3,0.0,0.0,0.0,118.9,5.0,76ers,0
3,New York Knicks,1960.0,27.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.765,70.0,22.2,0.0,0.0,0.0,117.3,5.0,Knicks,0
4,Atlanta Hawks,1960.0,46.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.745,71.2,25.1,0.0,0.0,0.0,113.4,10.0,Hawks,0
5,Detroit Pistons,1960.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.729,73.2,19.6,0.0,0.0,0.0,111.6,3.0,Pistons,0
6,Los Angeles Lakers,1960.0,25.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.730,72.4,19.3,0.0,0.0,0.0,107.3,7.0,Lakers,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,Denver Nuggets,2017.0,40.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.774,46.4,25.3,6.9,4.0,14.9,111.7,1.0,Nuggets,0
1321,New Orleans Pelicans,2017.0,34.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.750,43.7,22.8,7.8,5.5,12.9,104.3,5.0,Pelicans,0


In [85]:
data['pred'].sum()

49

In [88]:
len(data['Year'].unique())

58

In [95]:
data.iloc[1325]['All-Defensive']

1.0

In [96]:
data[data['pred'] == 1]

,Team,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,...,FT%,RPG,APG,STL,BLK,TOVPG,PPG,Accolades,Franchise ID,pred
0,Boston Celtics,1960.0,59.0,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.734,80.2,24.7,0.0,0.0,0.0,124.5,13.0,Celtics,1
8,Boston Celtics,1961.0,57.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.735,77.6,23.7,0.0,0.0,0.0,119.7,12.0,Celtics,1
16,Boston Celtics,1962.0,60.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.728,76.0,25.6,0.0,0.0,0.0,121.1,13.0,Celtics,1
25,Boston Celtics,1963.0,58.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.725,72.7,24.5,0.0,0.0,0.0,118.8,12.0,Celtics,1
34,Boston Celtics,1964.0,59.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.725,71.7,22.0,0.0,0.0,0.0,113.0,12.0,Celtics,1
43,Boston Celtics,1965.0,62.0,1,1.0,0.0,1.0,0.0,0.0,0.0,...,0.731,71.9,22.2,0.0,0.0,0.0,112.8,12.0,Celtics,1
53,Boston Celtics,1966.0,54.0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.739,69.9,22.4,0.0,0.0,0.0,112.7,10.0,Celtics,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,San Antonio Spurs,2007.0,58.0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.751,40.7,22.1,7.2,5.1,13.9,98.5,7.0,Spurs,1
1075,Los Angeles Lakers,2009.0,65.0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.770,43.9,23.3,8.8,5.1,13.5,106.9,6.0,Lakers,1


In [138]:
model.fit(data_train[data_train.Year != 2016][inputs], data_train[data_train.Year != 2016]['Champion'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [139]:
for i in range(1960,2017):
    print(i,model.predict_proba(data[data.Year == i][inputs]))

1960 [[ 0.1  0.9]
 [ 0.9  0.1]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]]
1961 [[ 0.2  0.8]
 [ 0.9  0.1]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 0.9  0.1]]
1962 [[ 0.   1. ]
 [ 0.8  0.2]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 0.9  0.1]
 [ 1.   0. ]]
1963 [[ 0.   1. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 0.9  0.1]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]]
1964 [[ 0.   1. ]
 [ 0.9  0.1]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 0.9  0.1]
 [ 0.9  0.1]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]]
1965 [[ 0.2  0.8]
 [ 1.   0. ]
 [ 0.9  0.1]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]]
1966 [[ 0.9  0.1]
 [ 0.4  0.6]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]]
1967 [[ 0.4  0.6]
 [ 0.7  0.3]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]]
1968 [[ 0.8  0.2]
 [ 0.2  0.8

In [247]:
inputs = ['Wins','MVP','Scoring Leader','Rebound Leader','Assist Leader',
          'WS Leader','DPOY','MIP','6MOY','Coach of Year','All-Stars',
          'All-Defensive','All-NBA','FG%','3P Attempts','3P%','2P Attempts',
          '2P%', 'FT%', 'RPG', 'APG', 'STL', 'BLK', 'TOVPG', 'PPG', 'Accolades']
model = RandomForestClassifier(n_estimators = 10)
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(1995,1998):
    #print(i)
    teams = []
    model.fit(data_train[data_train.Year != i][inputs], data_train[data_train.Year != i]['Champion'])
    predictions = model.predict_proba(data[data.Year == i][inputs])
    predictions = [m[1] for m in predictions]    
    index, value = max(enumerate(predictions), key=operator.itemgetter(1))
    
    predicted = data[data.Year == i].iloc[index]['Team']
    print('Predicted:',predicted)
    #print('Probability of Winning:', value*100 ,'%')
    actual = data[(data.Year == i) & (data.Champion == 1)].iloc[0]['Team']
    print('Actual:',actual)
    if predicted == actual:
        TP += 1
        TN += len(predictions)-1
    if predicted != actual:
        FP += 1
        FN += 1
        TN += len(predictions)-2
    #print('---')
    
print('TP:',TP)
print('FP:',FP)
print('TN:',TN)
print('FN:',FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
print('Precision:', precision)
print('Recall:', recall)
print('F1:', 1/((1/precision + 1/recall)/2))



Predicted: Orlando Magic
Actual: Houston Rockets
Predicted: Chicago Bulls
Actual: Chicago Bulls
Predicted: Chicago Bulls
Actual: Chicago Bulls
TP: 2
FP: 1
TN: 81
FN: 1
Precision: 0.6666666666666666
Recall: 1.0
F1: 0.8


In [333]:
def choose_from_two(pred1, pred2, year):
    elo1 = elo_data[(elo_data.year_id == year) & (elo_data.fran_id == pred1)]
    elo2 = elo_data[(elo_data.year_id == year) & (elo_data.fran_id == pred2)]

    if elo1['elo_i'].mean() > elo2['elo_i'].mean():
        return pred1
    return pred2

In [346]:
# Need to beat 30 TP, which is what you would get if you only bet first seed

In [348]:
inputs = ['Wins','MVP','Scoring Leader','Rebound Leader','Assist Leader',
          'WS Leader','MIP','6MOY','Coach of Year','All-Stars',
          'All-Defensive','All-NBA','FG%','3P Attempts','3P%',
          '2P%', 'FT%', 'RPG', 'APG', 'STL', 'BLK', 'TOVPG', 'PPG', 'Accolades'
         ]

model = RandomForestClassifier()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(1960,2018):
    #print(i)
    teams = []
    model.fit(data_train[data_train.Year != i][inputs], data_train[data_train.Year != i]['Champion'])
    predictions = model.predict_proba(data[data.Year == i][inputs])
    predictions = [m[1] for m in predictions]    
    
    index, value = max(enumerate(predictions), key=operator.itemgetter(1))
    predicted1 = data[data.Year == i].iloc[index]['Franchise ID']
    predictions[index] = 0
    index2, value2 = max(enumerate(predictions), key=operator.itemgetter(1))
    predicted2 = data[data.Year == i].iloc[index2]['Franchise ID']
    
    predicted = choose_from_two(predicted1, predicted2, i)
    
    
    actual = data[(data.Year == i) & (data.Champion == 1)].iloc[0]['Franchise ID']
    #print('Actual:',actual)
    if predicted == actual:
        TP += 1
        TN += len(predictions)-1
    else:
        FP += 1
        FN += 1
        TN += len(predictions)-2
    #print('---')
    
print('TP:',TP)
print('FP:',FP)
print('TN:',TN)
print('FN:',FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
print('Precision:', precision)
print('Recall:', recall)
print('F1:', 1/((1/precision + 1/recall)/2))



TP: 25
FP: 33
TN: 1236
FN: 33
Precision: 0.43103448275862066
Recall: 0.43103448275862066
F1: 0.43103448275862066


In [315]:
data[data.Year == 2016].iloc[-1]

Team              Boston Celtics
Year                        2016
Wins                          48
Champion                       0
MVP                            0
Scoring Leader                 0
Rebound Leader                 0
Assist Leader                  0
WS Leader                      0
DPOY                           0
MIP                            0
6MOY                           0
Coach of Year                  0
All-Stars                      1
All-Defensive                  1
All-NBA                        0
FG%                        0.439
3P Attempts                 2142
3P%                        0.335
2P Attempts                 5176
2P%                        0.483
FT%                        0.788
RPG                         44.9
APG                         24.2
STL                          9.2
BLK                          4.2
TOVPG                       13.7
PPG                        105.7
Accolades                      2
Franchise ID             Celtics
Name: 1296

In [272]:
from sklearn.dummy import DummyClassifier
model = DummyClassifier()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(1990,2018):
    #print(i)
    teams = []
    model.fit(data_train[data_train.Year != i][inputs], data_train[data_train.Year != i]['Champion'])
    predictions = model.predict_proba(data[data.Year == i][inputs])
    predictions = [m[1] for m in predictions]    
    index, value = max(enumerate(predictions), key=operator.itemgetter(1))
    
    predicted = data[data.Year == i].iloc[index]['Team']
    #print('Predicted:',predicted)
    #print('Probability of Winning:', value*100 ,'%')
    actual = data[(data.Year == i) & (data.Champion == 1)].iloc[0]['Team']
    #print('Actual:',actual)
    if predicted == actual:
        TP += 1
        TN += len(predictions)-1
    if predicted != actual:
        FP += 1
        FN += 1
        TN += len(predictions)-2
    #print('---')
    
print('TP:',TP)
print('FP:',FP)
print('TN:',TN)
print('FN:',FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
print('Precision:', precision)
print('Recall:', recall)
print('F1:', 1/((1/precision + 1/recall)/2))



TP: 3
FP: 25
TN: 762
FN: 25
Precision: 0.10714285714285714
Recall: 0.06
F1: 0.07692307692307693


In [297]:
reg_season = elo_data[elo_data.is_playoffs == 0]

In [298]:
elo_data[elo_data.]

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,40.294830,NYK,Knicks,68,1300.0000,1306.7233,H,L,0.640065,NaN
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,41.705170,TRH,Huskies,66,1300.0000,1293.2767,A,W,0.359935,NaN
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,42.012257,NYK,Knicks,47,1306.7233,1297.0712,H,W,0.631101,NaN
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,40.692783,CHS,Stags,63,1300.0000,1309.6521,A,L,0.368899,NaN
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,38.864048,WSC,Capitols,50,1300.0000,1320.3811,H,L,0.640065,NaN
5,3,194611020DTF,NBA,1,1947,11/2/1946,1,0,WSC,Capitols,...,43.135952,DTF,Falcons,33,1300.0000,1279.6189,A,W,0.359935,NaN
6,4,194611020PRO,NBA,1,1947,11/2/1946,1,0,BOS,Celtics,...,40.459381,PRO,Steamrollers,59,1300.0000,1305.1542,A,L,0.359935,NaN
7,4,194611020PRO,NBA,0,1947,11/2/1946,1,0,PRO,Steamrollers,...,41.540619,BOS,Celtics,53,1300.0000,1294.8458,H,W,0.640065,NaN
8,5,194611020STB,NBA,1,1947,11/2/1946,1,0,PIT,Ironmen,...,40.507980,STB,Bombers,56,1300.0000,1304.6908,A,L,0.359935,NaN
9,5,194611020STB,NBA,0,1947,11/2/1946,1,0,STB,Bombers,...,41.492020,PIT,Ironmen,51,1300.0000,1295.3092,H,W,0.640065,NaN
